<a href="https://colab.research.google.com/github/SwanMassart/MLF/blob/main/lab5_SwanMASSART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 11 08:59:13 2025

@author: Swan
"""
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
import tensorflow as tf
import matplotlib.pyplot as plt


X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y = np.array([[0], [1], [1], [0]], dtype=np.float32)

model = Sequential()
model.add(InputLayer(shape=(2,)))
model.add(Dense(2, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


history = model.fit(X, y, epochs=1000, batch_size=1, verbose=0)


loss, accuracy = model.evaluate(X, y, verbose=0)









"""


#  Expérimentation

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y = np.array([[0], [1], [1], [0]], dtype=np.float32)


epochs_list = [200, 1000, 5000]  # Nombre d'époques
learning_rates = [0.01, 0.1, 0.5]  # Taux d'apprentissage
activation_functions = ['sigmoid', 'relu', 'tanh']  # Activation
batch_sizes = [1, 4, 6]  # Batch size
num_neurons_list = [2, 4, 8]  # Nombre de neurones dans la couche cachée
verbose_mode = 0  # (0 = silencieux, 1 = affichage détaillé)


for epochs in epochs_list:
    for lr in learning_rates:
        for activation in activation_functions:
            for batch_size in batch_sizes:
                for num_neurons in num_neurons_list:

                    # Création du modèle
                    model = Sequential([
                        Input(shape=(2,)),
                        Dense(num_neurons, activation=activation),
                        Dense(1, activation='sigmoid')
                    ])

                    # Compilation avec l'optimiseur Adam
                    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
                    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

                    # Entraînement
                    history = model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=verbose_mode)

                    # Évaluation finale
                    loss, accuracy = model.evaluate(X, y, verbose=0)

                    # Résultats
                    print(f"\n Configuration testée :")
                    print(f"   Epochs: {epochs} |  Learning Rate: {lr} |  Activation: {activation}")
                    print(f"   Batch Size: {batch_size} |  Neurons: {num_neurons}")
                    print(f"  Final Accuracy: {accuracy * 100:.2f}%")

                    #  Affichage de la courbe de perte
                    plt.figure()
                    plt.plot(history.history['loss'])
                    plt.xlabel('Epochs')
                    plt.ylabel('Loss')
                    plt.title(f'Loss (Epochs={epochs}, LR={lr}, Act={activation}, Batch={batch_size}, Neurons={num_neurons})')
                    plt.show()
print('Accuracy: {:.2f}'.format(accuracy*100))

for id_x, data_sample in enumerate(X):
  prediction = model.predict(data_sample.reshape(1, -1))
  print(f"Data sample is {data_sample}, prediction from model {prediction}, ground_truth {y[id_x]}")


plt.figure()
plt.plot(history.history['loss'])
plt.xlabel('n epochs')
plt.ylabel('loss')
"""







#Exercise 2 - Congressional Voting Data







# -*- coding: utf-8 -*-
"""
Created on Tue Mar 11 09:54:18 2025

@author: Swan
"""

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


path_to_dataset = 'voting_complete.csv'
pd_dataset = pd.read_csv(path_to_dataset)

print(pd_dataset.head())





def custom_train_test_split(pd_data: pd.DataFrame, test_ratio: float = 0.2) -> tuple:
    pd_dataset = pd_data.copy()
    index = np.arange(len(pd_dataset))
    np.random.shuffle(index)

    train_amount = int(len(index) * (1 - test_ratio))
    train_ids, test_ids = index[:train_amount], index[train_amount:]

    train_dataset = pd_dataset.iloc[train_ids].reset_index(drop=True)
    test_dataset = pd_dataset.iloc[test_ids].reset_index(drop=True)

    X_train, y_train = train_dataset.iloc[:, 1:], train_dataset.iloc[:, 0]
    X_test, y_test = test_dataset.iloc[:, 1:], test_dataset.iloc[:, 0]

    return X_train, y_train, X_test, y_test


x_train, y_train, x_test, y_test = custom_train_test_split(pd_dataset)


x_train.replace({'y': 1, 'n': 0, '?': np.nan}, inplace=True)
x_test.replace({'y': 1, 'n': 0, '?': np.nan}, inplace=True)


x_train.fillna(x_train.mode().iloc[0], inplace=True)
x_test.fillna(x_test.mode().iloc[0], inplace=True)


x_train = pd.get_dummies(x_train, dtype=int)
x_test = pd.get_dummies(x_test, dtype=int)


x_test = x_test.reindex(columns=x_train.columns, fill_value=0)


y_train = y_train.replace({'Republican': 0, 'Democrat': 1})
y_test = y_test.replace({'Republican': 0, 'Democrat': 1})


model = Sequential()
model.add(Dense(32, input_shape=(x_train.shape[1],), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()



optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)  # Réduction du learning rate

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)




history = model.fit(X_train, y_train,
                    epochs=40,
                    batch_size=8,  # Augmentation du batch_size pour plus de stabilité
                    verbose=1,
                    validation_data=(X_val, y_val))

# Évaluation du modèle
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Model Accuracy: {accuracy * 100:.2f}%')





if 'loss' in history.history and 'val_loss' in history.history:
    # Création de la figure pour la perte
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.grid(True)
    plt.show()
else:
    print("Erreur ")

if 'accuracy' in history.history and 'val_accuracy' in history.history:
    # Création de la figure pour l'accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    plt.grid(True)
    plt.show()
else:
    print("Erreur ")